In [ ]:
import pdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import trange
from collections import defaultdict
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import clear_output

import itertools
from itertools import combinations, product
from itertools import groupby

from scipy.special import gamma
from scipy.spatial import distance
from scipy.spatial.distance import pdist, squareform

from sklearn import datasets
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets.samples_generator import make_blobs

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

from xgboost import XGBRegressor, XGBClassifier

from numba import jit
import threading

In [ ]:
class Lorenz():
    # Класс динамической системы Лоренца
    def __init__(self, sigma=10., beta=8./3., rho=28., start=0., end=100000., h=0.1):
        self.sigma = sigma
        self.beta = beta
        self.rho = rho
        self.start = start
        self.end = end
        self.h = h
        self.total = int(np.floor((end - start) / h))

    def dx(self, x, y, z): return self.sigma * (y - x)
    def dy(self, x, y, z): return x * (self.rho - z) - y
    def dz(self, x, y, z): return x * y - self.beta * z

    def Runge_Kutta(self, x, y, z, dx, dy, dz, h):
        # Метод интегрирования Рунге-Кутта 4-ого порядка
        k1_x, k1_y, k1_z = (f(x, y, z) for f in (dx, dy, dz))
        x_s, y_s, z_s = (r + 0.5 * h * k_r  for r, k_r in zip((x, y, z),(k1_x, k1_y, k1_z)))
        k2_x, k2_y, k2_z = (f(x_s, y_s, z_s) for f in (dx, dy, dz) )
        x_s, y_s, z_s = (r + 0.5 * h * k_r  for r, k_r in zip((x, y, z), (k2_x, k2_y, k2_z)))
        k3_x, k3_y, k3_z = (f(x_s, y_s, z_s) for f in (dx, dy, dz))
        x_s, y_s, z_s = (r + h * k_r  for r, k_r in zip((x, y, z), (k3_x, k3_y, k3_z)))
        k4_x, k4_y, k4_z  = ( f(x_s, y_s, z_s) for f in (dx, dy, dz))
        return  (r + (k1_r + 2 * k2_r + 2 * k3_r + k4_r) * (h / 6) for r, k1_r, k2_r, k3_r, k4_r in 
                zip((x, y, z), (k1_x, k1_y, k1_z), (k2_x, k2_y, k2_z), (k3_x, k3_y, k3_z), (k4_x, k4_y, k4_z)))

    def plot_series(self, values, left_border=0, right_border=-1):
        # Отрисовка временного ряда
        plt.figure(figsize=(20, 5))
        plt.plot(values[left_border:right_border])
        plt.ylabel('X')
        plt.xlabel('t')
        plt.show()

    def plot_attractor(self, x, y, z):
        # Отрисовка аттрактора
        fig = plt.figure()
        ax = Axes3D(fig)
        ax.set_xlabel("x")
        ax.set_ylabel("y")
        ax.set_zlabel("z")
        ax.set_title("Lorenz attractor (Runge-Kutta method)")
        ax.plot(x, y, z, color="red", lw=1)
        plt.show()
        plt.savefig("lorenz_attractor_runge_kutta.png")
    
    def normalize_series(self, data, scaler=MinMaxScaler()):
        # Нормализация ряда
        np_data = np.array(data).reshape(-1, 1)
        norm_data = scaler.fit_transform(np_data)
        return norm_data.reshape(1, -1).tolist()[0]

    def get_lorenz_series(self, normalize=True):
        # Получение временного ряда
        t = self.total * [0.0]
        x = self.total * [0.0]
        y = self.total * [0.0]
        z = self.total * [0.0]
        x[0],y[0],z[0],t[0] = 1., 1., 1., 0.

        for i in trange(1, self.total):
            x[i], y[i], z[i] = self.Runge_Kutta(x[i - 1], y[i - 1], z[i - 1], self.dx, self.dy, self.dz, self.h)

        print('\nTotal amount of steps:', self.total)
        
        if normalize:
            x = self.normalize_series(x)
            y = self.normalize_series(y)
            z = self.normalize_series(z)
        return x, y, z
    
def volume(r, m):
    return np.pi ** (m / 2) * r ** m / gamma(m / 2 + 1)

def significant(cluster, h, p):
    max_diff = max(abs(p[i] - p[j]) for i, j in product(cluster, cluster))
    return max_diff >= h

def get_clustering(x, k, h):
    print("Getting clusters...")
    n = len(x)
    m = len(x[0])
    dist = squareform(pdist(x))

    dk = []
    for i in trange(n):
        order = list(range(n))
        order.sort(key=lambda j: dist[i][j])
        dk.append(dist[i][order[k - 1]])

    p = [k / (volume(dk[i], m) * n) for i in range(n)]

    w = np.full(n, 0)
    completed = {0: False}
    last = 1
    vertices = set()
    for d, i in sorted(zip(dk, range(n))):
        neigh = set()
        neigh_w = set()
        clusters = defaultdict(list)
        for j in vertices:
            if dist[i][j] <= dk[i]:
                neigh.add(j)
                neigh_w.add(w[j])
                clusters[w[j]].append(j)

        vertices.add(i)
        if len(neigh) == 0:
            w[i] = last
            completed[last] = False
            last += 1
        elif len(neigh_w) == 1:
            wj = next(iter(neigh_w))
            if completed[wj]:
                w[i] = 0
            else:
                w[i] = wj
        else:
            if all(completed[wj] for wj in neigh_w):
                w[i] = 0
                continue
            significant_clusters = set(wj for wj in neigh_w if significant(clusters[wj], h, p))
            if len(significant_clusters) > 1:
                w[i] = 0
                for wj in neigh_w:
                    if wj in significant_clusters:
                        completed[wj] = (wj != 0)
                    else:
                        for j in clusters[wj]:
                            w[j] = 0
            else:
                if len(significant_clusters) == 0:
                    s = next(iter(neigh_w))
                else:
                    s = next(iter(significant_clusters))
                w[i] = s
                for wj in neigh_w:
                    for j in clusters[wj]:
                        w[j] = s
    return w

def make_template(series, train_beg, train_end, test_end, step1=1, step2=1, step3=1):
    # Нарезка ряда по шаблону
    delta = step1 + step2 + step3 + 1
    train_x = [[series[i], series[i + step1], series[i + step1 + step2], series[i + step1 + step2 + step3]] 
               for i in range(train_beg, train_end - delta)]
    test_x = [[series[i], series[i + step1], series[i + step1 + step2], series[i + step1 + step2 + step3]]
              for i in range(train_end - delta, test_end - delta)]
    return train_x, test_x

def get_centers(w):
    print("Getting centers...")
    centers = []
    sorted_by_cluster = sorted(range(len(w)), key=lambda x: w[x])
    for wi, cluster in groupby(sorted_by_cluster, lambda x: w[x]):
        cluster = list(cluster)
        center = np.full(4, 0.0)
        for i in cluster:
            center += train_x[i]
        centers.append(center / len(cluster))
    return centers

def apply_temp(cur_slice, template):
    sample = [cur_slice[0]]
    ind = 0
    for delta in template:
        ind += delta
        sample.append(cur_slice[ind])
    return sample

def get_scores(test_ts, centers, thr = 0.1, template = [1, 1, 1], steps_n = 1):
    predictable = [0] * steps_n
    scores = [0] * steps_n
    delta = sum(template) + 1
    for i in trange(len(test_ts) - delta - steps_n):
        cur_info = test_ts[i:i + delta]
        
        for k in range(steps_n):
            cur_temp = apply_temp(cur_info[-delta:], template)
            
            best_dist = 1e9
            for center in centers:
                cur_dist = distance.euclidean(cur_temp[:-1], center[:-1])
                if cur_dist <= thr and cur_dist < best_dist:
                    best_dist = distance.euclidean(cur_temp[:-1], center[:-1])
                    best_center = center

            if best_dist == 1e9:
                break

            predictable[k] += 1
            scores[k] += abs(best_center[-1] - cur_temp[-1])
            cur_info[-1] = best_center[-1]
            cur_info.append(test_ts[i + delta + k])
    return scores, predictable

def plot_all_metrics(all_metrics):
    for metrics in all_metrics:
        fig, ax = plt.subplots()
        ind = 1
        for metric in metrics:
            losses = []
            for i in range(len(metric[0])):
                if metric[1][i] > 10:
                    losses.append(metric[0][i] / metric[1][i])
            plt.plot(np.arange(1, len(losses) + 1), losses)
            plt.xticks(np.arange(1, 21, step = 1))

        plt.legend(thresholds)
        ax.set_ylabel('loss')
        ax.set_xlabel('steps')
        
def plot_final_metrics(scores_mae, scores_rmse, predictable_cnts, thr):
    legend = []
    for t in thr:
        legend.append(str(t))
        legend.append('daemon' + str(t))
    
    fig, ax = plt.subplots(figsize = (9, 9))
    for scores in scores_mae:
        plt.plot(np.arange(1, len(scores) + 1), scores)
    plt.xticks(np.arange(1, 11, step = 1))
    plt.grid(True)

    plt.legend(legend)
    ax.set_ylabel('MAE')
    ax.set_xlabel('steps')

    fig, ax = plt.subplots(figsize = (9, 9))
    for scores in scores_rmse:
        plt.plot(np.arange(1, len(scores) + 1), scores)
    plt.xticks(np.arange(1, 11, step = 1))
    plt.grid(True)

    plt.legend(legend)
    ax.set_ylabel('RMSE')
    ax.set_xlabel('steps')

    fig, ax = plt.subplots(figsize = (9, 9))
    for scores in predictable_cnts:
        plt.plot(np.arange(1, len(scores) + 1), 1000 - scores)
    plt.xticks(np.arange(1, 11, step = 1))
    plt.grid(True)

    plt.legend(legend)
    ax.set_ylabel('unpredictable')
    ax.set_xlabel('steps')

def get_final_scores(test_ts, result, n_steps = 10, is_optimized = False):
    scores_mae = []
    scores_rmse = []
    predictable_cnts = []
    daemon_scores_mae = []
    daemon_scores_rmse = []
    daemon_predictable_cnts = []
    for k in range(n_steps):
        score = 0
        score_rmse = 0
        predictable = 0
        daemon_score_mae = 0
        daemon_score_rmse = 0
        daemon_predictable = 0
        for i in range(len(test_ts)):
            total_counts = 0
            preds = []
            for j in range(len(result)):
                cur_pred = result[j][i][k]
                if cur_pred != -1:
                    preds.append(cur_pred)
                total_counts += 1
            if is_optimized:
                if len(preds) > 0 and max(preds) - min(preds) < 0.5:
                    predictable += 1
                    final_pred = sum(preds) / len(preds)
                    score += abs(final_pred - test_ts[i])
                    score_rmse += (final_pred - test_ts[i]) ** 2
                    if abs(final_pred - test_ts[i]) < 0.05:
                        daemon_score_mae += abs(final_pred - test_ts[i])
                        daemon_score_rmse += (final_pred - test_ts[i]) ** 2
                        daemon_predictable += 1
            else:
                if len(preds) > 0:
                    predictable += 1
                    final_pred = sum(preds) / len(preds)
                    score += abs(final_pred - test_ts[i])
                    score_rmse += (final_pred - test_ts[i]) ** 2
                    if abs(final_pred - test_ts[i]) < 0.05:
                        daemon_score_mae += abs(final_pred - test_ts[i])
                        daemon_score_rmse += (final_pred - test_ts[i]) ** 2
                        daemon_predictable += 1
                    
        scores_mae.append(score)
        scores_rmse.append(score_rmse)
        predictable_cnts.append(predictable)
        daemon_scores_mae.append(daemon_score_mae)
        daemon_scores_rmse.append(daemon_score_rmse)
        daemon_predictable_cnts.append(daemon_predictable)
        
    predictable_cnts = np.array(predictable_cnts)
    scores_mae = np.array(scores_mae)
    scores_rmse = np.array(scores_rmse)
    scores_mae /= predictable_cnts
    scores_rmse /= predictable_cnts
    scores_rmse = np.sqrt(scores_rmse)
    
    daemon_predictable_cnts = np.array(daemon_predictable_cnts)
    daemon_scores_mae = np.array(daemon_scores_mae)
    daemon_scores_rmse = np.array(daemon_scores_rmse)
    daemon_scores_mae /= daemon_predictable_cnts
    daemon_scores_rmse /= daemon_predictable_cnts
    daemon_scores_rmse = np.sqrt(daemon_scores_rmse)
    
    return scores_mae, scores_rmse, predictable_cnts, daemon_scores_mae, daemon_scores_rmse, daemon_predictable_cnts

def evaluate(test_ts, result, n_steps = 10, is_optimized = False):
    for k in range(n_steps):
        for i in range(len(test_ts)):
            total_counts = 0
            preds = []
            for j in range(len(result)):
                cur_pred = result[j][i][k]
                if cur_pred != -1:
                    preds.append(cur_pred)
                total_counts += 1
            if is_optimized:
                if len(preds) > 0 and max(preds) - min(preds) < 0.5:
                    predictable += 1
                    final_pred = sum(preds) / len(preds)
                    score += abs(final_pred - test_ts[i])
                    score_rmse += (final_pred - test_ts[i]) ** 2
                    if abs(final_pred - test_ts[i]) < 0.05:
                        daemon_score_mae += abs(final_pred - test_ts[i])
                        daemon_score_rmse += (final_pred - test_ts[i]) ** 2
                        daemon_predictable += 1
            else:
                if len(preds) > 0:
                    predictable += 1
                    final_pred = sum(preds) / len(preds)
                    score += abs(final_pred - test_ts[i])
                    score_rmse += (final_pred - test_ts[i]) ** 2
                    if abs(final_pred - test_ts[i]) < 0.05:
                        daemon_score_mae += abs(final_pred - test_ts[i])
                        daemon_score_rmse += (final_pred - test_ts[i]) ** 2
                        daemon_predictable += 1


def get_predictions(test_ts, center, thr = 1000, template = [1, 1, 1], steps_n = 1):
    delta = sum(template) + 1
    predictions = np.zeros((len(test_ts), steps_n)) - 1
#     predictions = [[[] for i in range(steps_n)] for j in range(len(test_ts))]
    best_distances = np.zeros((len(test_ts), steps_n)) - 1
    for i in trange(len(test_ts) - steps_n - delta):
        cur_info = test_ts[i:i + delta]
        flag = False

        for k in range(steps_n):
            cur_temp = apply_temp(cur_info[-delta:], template)

            best_dist = 1e9
            for center in centers:
                cur_dist = distance.euclidean(cur_temp[:-1], center[:-1])
                if cur_dist <= thr and cur_dist < best_dist:
                    best_dist = distance.euclidean(cur_temp[:-1], center[:-1])
                    best_center = center

            if best_dist == 1e9 or flag:
                predictions[i + delta + k][k] = -1
                flag = True
                continue

            predictions[i + delta + k][k] = best_center[-1]
            best_distances[i + delta + k][k] = best_dist
            cur_info[-1] = best_center[-1]
            cur_info.append(test_ts[i + delta + k])
    return predictions[delta + steps_n:], best_distances[delta + steps_n:]

def get_all_predictions(test_ts, center, thr = 1000, template = [1, 1, 1], steps_n = 1):
    delta = sum(template) + 1
    predictions = [[[] for i in range(steps_n)] for j in range(len(test_ts))]
    for i in trange(len(test_ts) - steps_n - delta):
        cur_info = test_ts[i:i + delta]
        flag = False

        for k in range(steps_n):
            cur_temp = apply_temp(cur_info[-delta:], template)

            best_dist = 1e9
            for center in centers:
                cur_dist = distance.euclidean(cur_temp[:-1], center[:-1])
                predictions[i + delta + k][k].append((cur_dist, center[-1]))
                if cur_dist <= thr and cur_dist < best_dist:
                    best_dist = distance.euclidean(cur_temp[:-1], center[:-1])
                    best_center = center

            if best_dist == 1e9 or flag:
                flag = True
                continue

            cur_info[-1] = best_center[-1]
            cur_info.append(test_ts[i + delta + k])
    return predictions[delta + steps_n:]

In [ ]:
lorenz = Lorenz(end=100000)
time_series = lorenz.get_lorenz_series()

In [ ]:
#Рассматриваемые промежутки для обучения и теста

train_beg = 0
train_end = 5000
test_end = 5500
# time_series = [x]

In [ ]:
# цикл по порогам и шаблонам

n_steps = 50
templates = []
for i in range(1, 4):
    for j in range(1, 4):
        for z in range(1, 4):
            templates.append((i, j, z))

results = []
dists = []
all_centers = []
for template in templates:
    print(template)
    delta = sum(template) + 1
    train_x, test_x = make_template(time_series[0], train_beg, train_end, test_end, 
                                    template[0], template[1], template[2])
    w = get_clustering(train_x, 11, 0.2)
    centers = get_centers(w)
    all_centers.append(centers)

In [ ]:
results = []
for i, template in enumerate(templates):
    print(template)
    delta = sum(template) + 1
    centers = all_centers[i]
    
    cur_preds = get_all_predictions(time_series[0][train_end - delta:test_end],
                                      centers, 1000, template, n_steps)
    results.append(cur_preds)

In [ ]:
def get_features(preds):
    preds.sort()
    
    dists = []
    predictions = []
    for i in range(len(preds)):
        dists.append(preds[i][0])
        predictions.append(preds[i][1])
    
    minmax_diff = max(predictions) - min(predictions)
    
    weighted_mean = 0
    w_sum = 0
    for i in range(len(preds)):
        weighted_mean += (1 - dists[i]) * predictions[i]
        w_sum += (1 - dists[i])
    weighted_mean /= w_sum

    mean_threshold = 0
    w_sum_thr = 0
    for i in range(len(preds)):
        if dists[i] > 0.05:
            continue
        mean_threshold += (1 - dists[i]) * predictions[i]
        w_sum_thr += (1 - dists[i])
    
    if w_sum_thr > 0:
        mean_threshold /= w_sum_thr
    else:
        mean_threshold = predictions[0]
    
    best_pred = predictions[0]
    
    return minmax_diff, weighted_mean, mean_threshold, best_pred

In [ ]:
train_size = 300
total_size = 400
n = total_size
m = len(results)

scores_mae = []
scores_rmse = []

dummy_mae = []
dummy_rmse = []

for k in range(0, n_steps):
    print(k)
    final = []
    for i in range(n):
        sample = []
        for j in range(m):
            feats = get_features(results[j][i][k])
            for feat in feats:
                sample.append(feat)
        final.append(sample)

    final = pd.DataFrame(final)
    X_train = final.loc[:train_size - 1]
    y_train = y_true[:train_size]
    X_test = final.loc[train_size:total_size]
    y_test = y_true[train_size:total_size]
    
    model = XGBRegressor()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    scores_mae.append(mean_absolute_error(y_test, preds))
    scores_rmse.append(mean_squared_error(y_test, preds, squared = False))